In [5]:
from basic_pitch.inference import predict, predict_and_save, Model
from basic_pitch import ICASSP_2022_MODEL_PATH, __file__
import tensorflow as tf
import pretty_midi
import math
import os
import csv
from pathlib import Path
from music21 import converter, interval, note, stream
from mido import MidiFile, MidiTrack

%load_ext autoreload
%autoreload 2

NUM_SONGS = 160

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
print(ICASSP_2022_MODEL_PATH)

c:\Users\aymli\anaconda3\envs\tomidi\lib\site-packages\basic_pitch\saved_models\icassp_2022\nmp


In [6]:
bp_root = os.path.dirname(__file__)
onnx_fp = os.path.join(bp_root, "saved_models",
                       "icassp_2022", "nmp.onnx")

In [17]:
print(Path(str(ICASSP_2022_MODEL_PATH) + "\saved_model.pb"))

c:\Users\aymli\anaconda3\envs\tomidi\lib\site-packages\basic_pitch\saved_models\icassp_2022\nmp\saved_model.pb


In [1]:
def difficulty(midi_path):
    pm = pretty_midi.PrettyMIDI(midi_path)

    beats = pm.get_beats()

    i = 0

    for inst in pm.instruments:
        for note in inst.notes:
            i += 1

    notes_per_beat = i / len(beats)

    diff = round(notes_per_beat/2, 2)

    return diff

def quantize(midi_path, subdivision=(4,3)):
        
    score = converter.parse(midi_path)
    score.flat.quantize(subdivision, processDurations=False)

    score = merge_notes(score)
    
    score.flat.write('midi', fp=midi_path)

def merge_notes(score):
    for part in score.parts:
        # Get all notes of this part, sorted by pitch then offset
        notes = sorted(part.flat.getElementsByClass(note.Note),
                       key=lambda n: (n.pitch.midi, n.offset))
        
        to_remove = []
        for i in range(len(notes) - 5):
            for j in range(i+1, i+4):
                n1, n2 = notes[i], notes[j]
                if n1.pitch == n2.pitch and (n1.offset + n1.quarterLength) > n2.offset:
                    
                    # They overlap: extend n1 to cover n2
                    new_end = max(n1.offset + n1.quarterLength,
                                n2.offset + n2.quarterLength)
                    n1.quarterLength = new_end - n1.offset
                    to_remove.append(n2)
        # Drop the shorter overlapping notes
        for orphan in to_remove:
            part.remove(orphan)
    return score

In [ ]:
basic_pitch_model = Model(onnx_fp)

with open("manifest.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["audio_filepath", "midi_filepath", "difficulty"])

for i in range(NUM_SONGS):
    try:
        predict_and_save(
            audio_path_list=[f"piano_data/{i}.mp3"],
            output_directory="piano_data",
            save_midi=True,
            sonify_midi=False,
            save_model_outputs=False,
            save_notes=False,
            model_or_model_path=basic_pitch_model,
            onset_threshold=0.3,
            frame_threshold=0.3,
            minimum_note_length = 150,
        )
        quantize(f"piano_data/{i}_basic_pitch.mid")
        diff = difficulty(f"piano_data/{i}_basic_pitch.mid")
        
        print(diff)
        with open("manifest.csv", "a", newline="") as f:
            writer = csv.writer(f)
            midi_filepath = fr"C:\Users\aymli\Cover\piano_data\{i}_basic_pitch.mid"
            audio_filepath = fr"C:\Users\aymli\Cover\song_data\{i}.mp3"
            writer.writerow([audio_filepath, midi_filepath, diff])
        
        #os.remove(f"piano_data/{i}.mp3")
    except:
        pass


Predicting MIDI for piano_data\0.mp3...


  Creating midi...
  💅 Saved to piano_data\0_basic_pitch.mid
4.37

Predicting MIDI for piano_data\1.mp3...


  Creating midi...
  💅 Saved to piano_data\1_basic_pitch.mid
3.62

Predicting MIDI for piano_data\2.mp3...


  Creating midi...
  💅 Saved to piano_data\2_basic_pitch.mid
3.69

Predicting MIDI for piano_data\3.mp3...


  Creating midi...
  💅 Saved to piano_data\3_basic_pitch.mid
3.99

Predicting MIDI for piano_data\4.mp3...


  Creating midi...
  💅 Saved to piano_data\4_basic_pitch.mid
4.62

Predicting MIDI for piano_data\5.mp3...


  Creating midi...

Predicting MIDI for piano_data\6.mp3...


  Creating midi...
  💅 Saved to piano_data\6_basic_pitch.mid
4.54

Predicting MIDI for piano_data\7.mp3...
